# Imports

In [4]:
from utils import ixbrl_frs_102_2024
from bs4 import BeautifulSoup
from document_retrieval import *
import re

# Functions

In [5]:
def retrieve_all_ix_financial_data_minus(xhtml_content_path):
    """
    Retrieve financial values from the XHTML content, including detecting negative values and adjusting for decimals.
    
    Args:
        xhtml_content_path (str): The path to the XHTML file.
    
    Returns:
        dict: A dictionary containing the financial values.
    """

    # Open the XHTML file in read mode with UTF-8 encoding
    with open(xhtml_content_path, 'r', encoding='utf-8') as file:
        # Read the entire content of the file into a string variable
        xhtml_content = file.read()

    # Parse the XHTML content using BeautifulSoup with 'html.parser' to create a soup object
    soup = BeautifulSoup(xhtml_content, 'html.parser')
    
    # Initialize an empty dictionary to store the financial values
    dict_name = {}

    # Iterate over each label in the ixbrl_labels_2024_full list
    for label in ixbrl_frs_102_2024:
        
        # Check if the label is not empty or None
        if label:
            print(f'label: {label}')
            # Select all elements in the soup that have a 'name' attribute ending with the current label
            # elements = soup.select(f'[name$="{label}"]')
            elements = soup.select(f'[name$=":{label}"]')
            # Initialize an empty list to temporarily store the values for the current label
            temp_list = []
            # Iterate over each element found
            for item in elements:
                print(f'item: {item}')
                # Extract and strip the text content of the element
                value = item.text.strip()
                # Check if the element has a 'sign' attribute and if its value is '-'
                if item.has_attr('sign') and item['sign'] == '-':
                    # Prepend a '-' to the value to indicate it is negative
                    value = '-' + value
                
                # Check if the element has a 'decimals' attribute and if its value is '-3'
                if item.has_attr('decimals') and item['decimals'] == '-3':
                    # Convert the value to an integer (removing commas), multiply by 1000, and format with commas
                    # This is because '-3' means they simplify the reporting and values need to be multiplied by 1000
                    value = '{:,}'.format(int(value.replace(',', '')) * 1000)
                
                # Append the processed value to the temporary list
                temp_list.append(value)
                print(f'temp list: {temp_list}')

            # Assign the temporary list to the dictionary with the current label as the key
            dict_name[label] = temp_list

    # Return the dictionary containing all the financial values
    return dict_name

### Micro, Small or Medium selecting criteria

Data obtaine from:   
https://www.gov.uk/government/publications/life-of-a-company-annual-requirements/life-of-a-company-part-1-accounts

Micro companies
- a turnover of £632,000 or less
- £316,000 or less on its balance sheet
- 10 employees or less


Qualifying as a small company   
For accounting periods beginning on or after 1 January 2016, a small company must meet at least 2 of the following conditions:   

- annual turnover must be not more than £10.2 million
- the balance sheet total must be not more than £5.1 million
- the average number of employees must be not more than 50

Qualifying as a medium-sized company
To be a medium-sized company, you must meet at least 2 of the following conditions:

- the annual turnover must be no more than £36 million
- the balance sheet total must be no more than £18 million
- the average number of employees must be no more than 250


#### company_size() Function   

This function determines the size of a company based on the UK Company House criteria by retrieving and processing relevant ixbrl labels from an XHTML file. The function follows these steps:

1. **Define Criteria**: It defines the criteria for different company sizes (micro, small, medium) based on turnover, balance sheet total, and number of employees.

2. **Retrieve ixbrl Data**: It retrieves financial data from the XHTML file using the `retrieve_all_ix_financial_data_minus` function.

3. **Extract Data**: It extracts the required data (turnover, fixed assets, current assets, employees) from the retrieved ixbrl data.

4. **Handle Missing Data**:
   - **Turnover**: If turnover data is missing, it attempts to retrieve it directly from the XHTML content by searching for specific tags.
   - **Fixed Assets**: If fixed assets data is missing, it calculates the value by aggregating values from its subcomponents.
   - **Current Assets**: If current assets data is missing, it calculates the value by aggregating values from its subcomponents.
   - **Employees**: If employee data is missing, it logs a message.

5. **Convert Data**: It converts the extracted string values to integers for numerical processing.

6. **Calculate Balance Sheet**: It calculates the total balance sheet value for the latest and previous year by summing fixed and current assets.

7. **Format Data**: It formats the numerical values with commas for human readability.

8. **Determine Company Size**: It determines the size of the company by checking if it meets at least 2 out of 3 conditions for each size category (micro, small, medium). If none of these conditions are met, the company is classified as large.

The function also includes detailed logging to help trace the steps and identify any issues during execution.

In [6]:
def company_size(file_path):
    """
    Determine the size of a company based on the UK Company House criteria.

    Args:
        file_path (str): The path to the XHTML file.

    Returns:
        str: The size of the company ('micro', 'small', 'medium', or 'large').
    """
    # Define the criteria for different company sizes
    micro_criteria = {
        'turnover': 632000,
        'balance_sheet': 316000,
        'employees': 10
    }
    small_criteria = {
        'turnover': 10200000,
        'balance_sheet': 5100000,
        'employees': 50
    }
    medium_criteria = {
        'turnover': 36000000,
        'balance_sheet': 18000000,
        'employees': 250
    }

    # Retrieve ixbrl data from the XHTML file
    ixbrl_data = retrieve_all_ix_financial_data_minus(file_path)

    # Extract the required data from the ixbrl_data dictionary
    turnover = ixbrl_data.get('TurnoverRevenue', [])
    fixed_assets = ixbrl_data.get('FixedAssets', [])
    current_assets = ixbrl_data.get('CurrentAssets', [])
    employees = ixbrl_data.get('AverageNumberEmployeesDuringPeriod', [])

    # Check if any of the ixbrl labels values list is empty and display a message
    if not turnover:
        print("TurnoverRevenue list is empty.")
        # Retrieve turnover values from the XHTML content if not present in ixbrl_data
        with open(file_path, 'r') as file:
            content = file.read()
            print(f'attempting to match Turnover')
            
            # Find the tag containing 'Turnover' or 'turnover'
            turnover_start = re.search(r'<div[^>]*>Turnover</div>|<div[^>]*>turnover</div>', content, re.IGNORECASE)
            if turnover_start:
                print(f"Found 'Turnover' tag")
                # Get the content after the 'Turnover' tag
                content_after_turnover = content[turnover_start.end():]
                
                # Find the next monetary values, excluding single digits other than zero
                turnover_values = re.findall(r'>(\d{2,}|\d{1,3}(?:,\d{3})+)</div>', content_after_turnover)
                if turnover_values:
                    turnover = [value.replace(',', '') for value in turnover_values[:2]]
                    print(f"Turnover values retrieved from XHTML content: {turnover}")
                else:
                    print("Could not find valid monetary values after 'Turnover' tag")
            else:
                print("Could not find 'Turnover' tag")

    if not fixed_assets:
        print("FixedAssets list is empty.")
    if not current_assets:
        print("CurrentAssets list is empty.")
    if not employees:
        print("AverageNumberEmployeesDuringPeriod list is empty.")

    # Convert string values to integers after removing commas
    turnover = [int(value.replace(',', '')) for value in turnover]
    employees = [int(value.replace(',', '')) for value in employees]

    # Calculate fixed assets if it's empty
    if not fixed_assets:
        fixed_assets_components = [
            'IntangibleAssets',
            'PropertyPlantEquipment',
            'InvestmentFixedAssets',
            'InvestmentProperty',
            'BiologicalAssetsNon-current'
        ]
        
        fixed_assets = [0, 0]  # Initialize with two zeros for the last two years
        for component in fixed_assets_components:
            component_values = ixbrl_data.get(component, [])
            if component_values:
                for i, value in enumerate(component_values[:2]):  # Use only the first two items that generally represent the total sum of the subcomponents
                    try:
                        int_value = int(value.replace(',', ''))
                        fixed_assets[i] += int_value
                    except ValueError:
                        print(f"ValueError: invalid literal for int() with base 10: '{value}' in label '{component}'")
        
        # Calculate 'InvestmentFixedAssets' if it's empty
        if not ixbrl_data.get('InvestmentFixedAssets', []):
            investment_fixed_assets_components = [
                'InvestmentsInGroupUndertakings',
                'LoansToGroupUndertakings',
                'InvestmentsInAssociatesJointVenturesParticipatingInterests',
                'LoansToAssociatesJointVenturesParticipatingInterests',
                'OtherInvestmentsOtherThanLoans',
                'OtherLoansClassifiedUnderInvestments',
                'OwnShares'
            ]
            investment_fixed_assets = [0, 0]  # Initialize with two zeros for the last two years
            for component in investment_fixed_assets_components:
                component_values = ixbrl_data.get(component, [])
                if component_values:
                    for i, value in enumerate(component_values[:2]):  # Use only the first two items that generally represent the total sum of the subcomponents
                        try:
                            int_value = int(value.replace(',', ''))
                            investment_fixed_assets[i] += int_value
                        except ValueError:
                            print(f"ValueError: invalid literal for int() with base 10: '{value}' in label '{component}'")
            fixed_assets = [fa + ifa for fa, ifa in zip(fixed_assets, investment_fixed_assets)]
            print("Investment Fixed Assets value was missing and calculated using components.")
        print("Fixed Assets value was missing and calculated using components.")
    else:
        fixed_assets = [int(value.replace(',', '')) for value in fixed_assets[:2]]  # Use only the first two items

    # Calculate current assets if it's empty
    if not current_assets:
        current_assets_components = [
            'TotalInventories',
            'BiologicalAssetsCurrent',
            'Debtors',
            'CurrentAssetInvestments',
            'CashBankOnHand'
        ]
        
        current_assets = [0, 0]  # Initialize with two zeros for the last two years
        for component in current_assets_components:
            component_values = ixbrl_data.get(component, [])
            if component_values:
                for i, value in enumerate(component_values[:2]):  # Use only the first two items that generally represent the total sum of the subcomponents
                    try:
                        int_value = int(value.replace(',', ''))
                        current_assets[i] += int_value
                    except ValueError:
                        print(f"ValueError: invalid literal for int() with base 10: '{value}' in label '{component}'")
        
        # Calculate 'Debtors' if it's empty
        if not ixbrl_data.get('Debtors', []):
            debtors_components = [
                'TradeDebtorsTradeReceivables',
                'AmountsOwedByGroupUndertakings',
                'AmountsOwedByAssociatesJointVenturesParticipatingInterests',
                'OtherDebtorsBalanceSheetSubtotal',
                'CalledUpShareCapitalNotPaidCurrentAsset',
                'PrepaymentsAccruedIncome',
                'DeferredTaxAssetDebtors'
            ]
            debtors = [0, 0]  # Initialize with two zeros for the last two years
            for component in debtors_components:
                component_values = ixbrl_data.get(component, [])
                if component_values:
                    for i, value in enumerate(component_values[:2]):  # Use only the first two items that generally represent the total sum of the subcomponents
                        try:
                            int_value = int(value.replace(',', ''))
                            debtors[i] += int_value
                        except ValueError:
                            print(f"ValueError: invalid literal for int() with base 10: '{value}' in label '{component}'")
            current_assets = [ca + d for ca, d in zip(current_assets, debtors)]
            print("Debtors value was missing and calculated using components.")
        
        # Calculate 'CurrentAssetInvestments' if it's empty
        if not ixbrl_data.get('CurrentAssetInvestments', []):
            current_asset_investments_components = [
                'InvestmentsInGroupUndertakings',
                'OwnShares',
                'OtherCurrentAssetInvestmentsBalanceSheetSubtotal'
            ]
            current_asset_investments = [0, 0]  # Initialize with two zeros for the last two years
            for component in current_asset_investments_components:
                component_values = ixbrl_data.get(component, [])
                if component_values:
                    for i, value in enumerate(component_values[:2]):  # Use only the first two items that generally represent the total sum of the subcomponents
                        try:
                            int_value = int(value.replace(',', ''))
                            current_asset_investments[i] += int_value
                        except ValueError:
                            print(f"ValueError: invalid literal for int() with base 10: '{value}' in label '{component}'")
            current_assets = [ca + cai for ca, cai in zip(current_assets, current_asset_investments)]
            print("Current Asset Investments value was missing and calculated using components.")
        print("Current Assets value was missing and calculated using components.")
    else:
        current_assets = [int(value.replace(',', '')) for value in current_assets[:2]]  # Use only the first two items

    # Calculate the total balance sheet value for the latest and previous year
    balance_sheet_latest = sum(fixed_assets[-2:-1]) + sum(current_assets[-2:-1])
    balance_sheet_previous = sum(fixed_assets[-1:]) + sum(current_assets[-1:])
    balance_sheet = [balance_sheet_latest, balance_sheet_previous]

    # Convert values to strings with commas for human readability
    turnover = [f"{value:,}" for value in turnover]
    fixed_assets = [f"{value:,}" for value in fixed_assets]
    current_assets = [f"{value:,}" for value in current_assets]
    employees = [f"{value:,}" for value in employees]
    balance_sheet = [f"{value:,}" for value in balance_sheet]

    # Print values in the same column
    print('-'*70)
    print(f"{'Turnover Revenue:':<35} {turnover}")
    print(f"{'Fixed Assets:':<35} {fixed_assets}")
    print(f"{'Current Assets:':<35} {current_assets}")
    print(f"{'Average Number Employees | Period:':<35} {employees}")
    print(f"{'Balance Sheet Total:':<35} {balance_sheet}")
    print('-'*70)

    # Determine the size of the company based on the criteria
    # Check if the company meets at least 2 out of 3 conditions for micro size
    micro_conditions_met = (
        (len(turnover) > 1 and int(turnover[-2].replace(',', '')) <= micro_criteria['turnover']) +
        (int(balance_sheet[-2].replace(',', '')) <= micro_criteria['balance_sheet']) +
        (len(employees) > 1 and int(employees[-2].replace(',', '')) <= micro_criteria['employees'])
    )
    if micro_conditions_met >= 2:
        print(f"Size of Entity: Micro")
        return 'micro'

    # Check if the company meets at least 2 out of 3 conditions for small size
    small_conditions_met = (
        (len(turnover) > 1 and int(turnover[-2].replace(',', '')) <= small_criteria['turnover']) +
        (int(balance_sheet[-2].replace(',', '')) <= small_criteria['balance_sheet']) +
        (len(employees) > 1 and int(employees[-2].replace(',', '')) <= small_criteria['employees'])
    )
    if small_conditions_met >= 2:
        print(f"Size of Entity: Small")
        return 'small'

    # Check if the company meets at least 2 out of 3 conditions for medium size
    medium_conditions_met = (
        (len(turnover) > 1 and int(turnover[-2].replace(',', '')) <= medium_criteria['turnover']) +
        (int(balance_sheet[-2].replace(',', '')) <= medium_criteria['balance_sheet']) +
        (len(employees) > 1 and int(employees[-2].replace(',', '')) <= medium_criteria['employees'])
    )
    if medium_conditions_met >= 2:
        print(f"Size of Entity: Medium")
        return 'medium'

    # If none of the above conditions are met, the company is classified as large
    print(f"Size of Entity: Large")
    return 'large'


# Load data

In [7]:
import pandas as pd

path = "csv/UK_SMB_manually_selected_set100.csv"
data = pd.read_csv(path)

data_ixbrl = data[data['ixbrl'] == 1] #select only the rows where ixbrl == 1 (companies that contain xhtml in their filing)

data_ixbrl = data_ixbrl[['company_number', 'company_name', 'ixbrl', 'size']]
data_ixbrl

,company_number,company_name,ixbrl,size
7,11717,ULVERSTON AUCTION MART PUBLIC LIMITED COMPANY(...,1,small
15,1024277,TOTAL SYSTEMS PLC,1,small
19,737075,SANDUSKY LIMITED,1,small
20,277871,ALAN HADLEY LIMITED,1,small
23,2847809,NORSK EUROPEAN WHOLESALE LIMITED,1,medium
27,114709,J F ASHTON LIMITED,1,medium
30,1349434,PSD LIMITED,1,medium
31,335313,COLE & SON (WALLPAPERS) LIMITED,1,medium
33,650255,TORPEDO FACTORY LTD,1,small
34,865285,VISTRA LIMITED,1,small


# Evaluate

## Notes

The benchmark was againt size classification made by Endole.   
There are cases when the Endole class is outdated, for example for company: G.B. KENT & SONS PLC, currently 'small'.   

In [8]:
sizes = []

print("Starting to process companies...")

for index, row in data_ixbrl.iterrows():
    
    print('#'*100)
    code = str(row['company_number']).zfill(8)
    company_name = row['company_name']
    print(f"\nProcessing company: {company_name} (ID: {code})")

    history = request_filling_history(code)
    print(f"Retrieved filling history for {company_name}")

    filtered_history = [item for item in history['items'] if item.get('type') == 'AA']

    if not filtered_history:
        error_message = f"No 'AA' type items found in history for company ID: {code}"
        print(error_message)
        sizes.append(None)
        continue

    print(f"Found {len(filtered_history)} 'AA' type items for {company_name}")

    for item in filtered_history:
        if check_for_xhtml_pipe(item):
            print(f"XHTML document found for {company_name}")
            
            # Retrieve the URL of the documents
            url = retrieve_documents_url(item)
            print(f"Retrieved document URL for {company_name}")

            # Retrieve the json object containing the documents (pdf + xhtml possibly)
            response = make_get_request(url, headers=None)
            url = response.json()["links"]["document"]
            print(f"Retrieved JSON object for {company_name}")

            # Retrieve the desired document
            headers = {"Accept": "application/xhtml+xml"}
            response = make_get_request(url, headers)
            print(f"Retrieved XHTML document for {company_name}")
            
            # The response should be a binary of the xhtml file
            xhtml_content = response.content.decode("utf-8")

            # Save the xhtml file with company_number, date_str, type_str, and company_name in the file name
            date_str = retrieve_date_from_accounts_item(item)
            type_str = retrieve_type_from_accounts_item(item)
            file_name = f"{code}_{type_str}_{date_str}_{company_name}.xhtml"
            output_directory = "xhtml"
            if not os.path.exists(output_directory):
                os.makedirs(output_directory)
            output_file_path = os.path.join(output_directory, file_name)

            with open(output_file_path, "w", encoding="utf-8") as file:
                file.write(xhtml_content)
            print(f"Saved XHTML file for {company_name}")

            # Classify size
            print(f"Classifying size for {company_name}...")
            # ixbrl_data = retrieve_all_ix_financial_data_minus(output_file_path)
            size = company_size(output_file_path)
            print(f"Company: {company_name}, Size: {size}")
            sizes.append(size)
            break
    else:
        print(f"No XHTML document found for {company_name}")
        sizes.append(None)

print("\nFinished processing all companies.")
# Add the 'size' column to the existing DataFrame
data_ixbrl['size evals'] = sizes
print("Added size data to dataframe.")


Starting to process companies...
####################################################################################################

Processing company: ULVERSTON AUCTION MART PUBLIC LIMITED COMPANY(THE) (ID: 00011717)
Retrieved filling history for ULVERSTON AUCTION MART PUBLIC LIMITED COMPANY(THE)
Found 8 'AA' type items for ULVERSTON AUCTION MART PUBLIC LIMITED COMPANY(THE)
XHTML document found for ULVERSTON AUCTION MART PUBLIC LIMITED COMPANY(THE)
Retrieved document URL for ULVERSTON AUCTION MART PUBLIC LIMITED COMPANY(THE)
Retrieved JSON object for ULVERSTON AUCTION MART PUBLIC LIMITED COMPANY(THE)
Retrieved XHTML document for ULVERSTON AUCTION MART PUBLIC LIMITED COMPANY(THE)
Saved XHTML file for ULVERSTON AUCTION MART PUBLIC LIMITED COMPANY(THE)
Classifying size for ULVERSTON AUCTION MART PUBLIC LIMITED COMPANY(THE)...
label: TurnoverRevenue
item: <ix:nonfraction contextref="c0" decimals="0" format="ixt:numcommadot" name="e:TurnoverRevenue" unitref="u1">673,534</ix:nonfraction>

# Save results to CSV

A spreadsheet with evaluations results and notes can be found here:    
https://docs.google.com/spreadsheets/d/1aMZyXyjDJWzqOB2Jste8AaunHuYLa6t9uy1VnB81g5c/edit?gid=0#gid=0

In [9]:
# Save data_ixbrl to CSV with timestamp
import datetime

# Get current timestamp in human-readable format
timestamp = datetime.datetime.now().strftime("%Y-%m-%d_%H-%M-%S")

# Create filename with timestamp
filename = f"csv/results_data_ixbrl_{timestamp}.csv"

# Save DataFrame to CSV
data_ixbrl.to_csv(filename, index=False)

print(f"Data saved to {filename}")

# Display the DataFrame
data_ixbrl

Data saved to csv/results_data_ixbrl_2024-07-04_11-53-01.csv


,company_number,company_name,ixbrl,size,size evals
7,11717,ULVERSTON AUCTION MART PUBLIC LIMITED COMPANY(...,1,small,small
15,1024277,TOTAL SYSTEMS PLC,1,small,small
19,737075,SANDUSKY LIMITED,1,small,small
20,277871,ALAN HADLEY LIMITED,1,small,small
23,2847809,NORSK EUROPEAN WHOLESALE LIMITED,1,medium,medium
27,114709,J F ASHTON LIMITED,1,medium,medium
30,1349434,PSD LIMITED,1,medium,medium
31,335313,COLE & SON (WALLPAPERS) LIMITED,1,medium,medium
33,650255,TORPEDO FACTORY LTD,1,small,small
34,865285,VISTRA LIMITED,1,small,small
